In [1]:
!pip install h2o

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datetime import datetime
import pandas as pd
import json
import os
import re

# from azure.storage.blob import BlobClient

import h2o
from h2o.automl import H2OAutoML

In [40]:
j = open('param_h2o.json', 'r').read()
param = json.loads(j)
# param

In [4]:
source_path = param['sourcePath']
saving_path = param['savingPath']
target = param['target']
max_models = int(param['maxModels'])
seed = int(param['seed'])
max_training_time = int(param['maxTrainingTime'])

In [5]:
h2o.init()
# h2o.init(ip = "192.168.59.112", port = 54321)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312"; OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~20.04-b07); OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Starting server from /home/hobbes/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprmq7pqbx
  JVM stdout: /tmp/tmprmq7pqbx/h2o_hobbes_started_from_python.out
  JVM stderr: /tmp/tmprmq7pqbx/h2o_hobbes_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_hobbes_n3yrdp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.703 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [6]:
df = h2o.import_file(path=source_path)

classification = list(df[target].types.values())[0] == 'enum'

if classification:
    df[target] = df[target].asfactor()

train, test = df.split_frame(ratios=[0.8], seed = seed)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
aml = H2OAutoML(
    max_models = max_models, 
    max_runtime_secs = max_training_time,  
    balance_classes=True, 
    seed = seed
)

aml.train(training_frame = train, y = target)

AutoML progress: |
23:43:17.342: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

██████████████
23:43:31.381: _train param, Dropping bad and constant columns: [merchant_country, merchant_id, status_id, customer_id, delivery_address_country, order_id]

█████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220123_234317


Model Summary: 


,,number_of_trees
0,,37.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.20007016732161934
RMSE: 0.44729203807089984
LogLoss: 0.586563963560127
Mean Per-Class Error: 0.5
AUC: 0.7816557796981027
AUCPR: 0.9991408825275204
Gini: 0.5633115593962055

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.11575616151094437: 


,,False,True,Error,Rate
0,False,0.0,99.0,1.0,(99.0/99.0)
1,True,0.0,32181.0,0.0,(0.0/32181.0)
2,Total,0.0,32280.0,0.0031,(99.0/32280.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.996933,0.000482,0.996437,0.997212,0.997522,0.997057,0.996437
1,auc,0.684544,0.054854,0.707794,0.627520,0.623966,0.723887,0.739553
2,err,0.003067,0.000482,0.003563,0.002788,0.002478,0.002943,0.003563
3,err_count,19.800000,3.114482,23.000000,18.000000,16.000000,19.000000,23.000000
4,f0point5,0.997545,0.000386,0.997148,0.997768,0.998016,0.997644,0.997148
5,f1,0.998464,0.000242,0.998216,0.998604,0.998759,0.998526,0.998216
6,f2,0.999385,0.000097,0.999285,0.999441,0.999503,0.999410,0.999285
7,lift_top_group,1.003077,0.000485,1.003575,1.002796,1.002484,1.002952,1.003575
8,logloss,0.606751,0.020484,0.580044,0.601029,0.634858,0.617067,0.600759
9,max_per_class_error,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-01-24 00:43:28,11.296 sec,0.0,0.500000,0.693147,0.500000,0.996933,1.000000,0.003067
1,,2022-01-24 00:43:29,11.850 sec,5.0,0.476128,0.646044,0.672359,0.998294,1.002066,0.003067
2,,2022-01-24 00:43:29,12.028 sec,10.0,0.452516,0.600318,0.705753,0.998636,1.003076,0.003067
3,,2022-01-24 00:43:29,12.259 sec,15.0,0.456109,0.606174,0.718651,0.998677,1.003076,0.003067
4,,2022-01-24 00:43:29,12.427 sec,20.0,0.452172,0.598146,0.744456,0.998866,1.003076,0.003067
5,,2022-01-24 00:43:29,12.585 sec,25.0,0.454337,0.601947,0.753145,0.998967,1.003076,0.003067
6,,2022-01-24 00:43:30,12.737 sec,30.0,0.454331,0.601295,0.764283,0.999018,1.003076,0.003067
7,,2022-01-24 00:43:30,12.926 sec,35.0,0.443748,0.579918,0.778351,0.999122,1.003076,0.003067
8,,2022-01-24 00:43:30,13.035 sec,37.0,0.447292,0.586564,0.781656,0.999141,1.003076,0.003067



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,status_created_at,15.591026,1.000000,0.174366
1,customer_phone_number,12.366129,0.793157,0.138300
2,delivery_address_external_id,10.984626,0.704548,0.122850
3,merchant_zip_code,9.642291,0.618451,0.107837
4,delivery_address_latitude,8.373600,0.537078,0.093648
5,delivery_address_zip_code,6.056512,0.388461,0.067735
6,delivery_address_longitude,4.789057,0.307168,0.053560
7,minimum_order_value,4.662842,0.299072,0.052148
8,cpf,4.393958,0.281826,0.049141
9,merchant_latitude,4.085164,0.262020,0.045688


In [8]:
best_model = aml.get_best_model()

model_info = best_model.model_performance(test)

In [9]:
local_path = os.getcwd()+'/'

In [10]:
model_path_1 = h2o.save_model(model=best_model,path=local_path, force=True)

In [11]:
best_model = model_path_1.split('/')[int(len(model_path_1.split('/'))-1)]

model_id = best_model.split('_')[-2]+best_model.split('_')[-1]
model_name = best_model.split('_')[0]+'_'+model_id
model_date = best_model.split('_')[-2]

In [12]:
test_name = 'test_' + model_id
train_name = 'train_' + model_id

In [13]:
test.as_data_frame().convert_dtypes().to_pickle(test_name + '.csv')
train.as_data_frame().convert_dtypes().to_pickle(train_name + '.csv')

In [14]:
model_path = saving_path + model_name
    
info1 = {
        "id" : str(model_id),
        "name" : str(model_name),
        "date" : datetime. strptime(model_date, '%Y%m%d'),
        "h2o_version" : str(h2o.__version__),
        "seed" : str(seed),
        "target" : str(target),
        "classification" : str(classification),
        "max_runtime" : str(aml.max_runtime_secs),
        "mse" : str(model_info.mse()),
        "rmse" : str(model_info.rmse()),
        "train_path" : str(saving_path + test_name+ '.csv'),
        "test_path" : str(saving_path + train_name+ '.csv'),
        "model_path" : str(saving_path + model_name),
        "columns" : str(list(aml.varimp().index)),
        "varimp" : str(aml.varimp().to_json())
}


if classification == False:
    info2 = {
        "metrics":{
            "mae":str(model_info.mae()),
            "rmsle":str(model_info.rmsle()),
            "r2":str(model_info.r2()),
            "mean_resid_deviance":str(model_info.mean_residual_deviance())
        }
    }

if classification == True:
    try:
        accuracy = str(model_info.accuracy()[0])
        recall = str(model_info.recall()[0])
        precision = str(model_info.precision()[0])
        f0point5 = str(model_info.F0point5()[0])
        f1 = str(model_info.F1()[0])
        f2 = str(model_info.F2()[0])
        error = str(model_info.error()[0])
        max_per_class_error = str(model_info.max_per_class_error()[0])
        specificity = str(model_info.specificity()[0])
    
    except:
        accuracy = str('NULL')
        recall = str('NULL')
        precision = str('NULL')
        f0point5 = str('NULL')
        f1 = str('NULL')
        f2 = str('NULL')
        error = str('NULL')
        max_per_class_error = str('NULL')
        specificity = str('NULL')
        
    info2 = {
        "metrics":{          
            "accuracy" : accuracy,
            "recall" : recall,
            "precision" : precision,
            "f0point5" : f0point5,
            "f1" : f1,
            "f2" : f2,
            "auc" : str(model_info.auc()),
            "error" : error,
            "logloss" : str(model_info.logloss()),
            "max_per_class_error" : max_per_class_error,
            "mean_per_class_accuracy" : str(),
            "mean_per_class_error" : str(model_info.mean_per_class_error()[0]),
            "aucpr" : str(model_info.aucpr()),
            "r2" : str(model_info.r2()),
            "specificity" : specificity
        }
    }

    
info = info1.copy()
info.update(info2)

In [15]:
info_new = pd.DataFrame([info])

info_path = local_path + 'info.csv'

try:
    info_old = pd.read_csv(info_path)
    if info_old.columns[0] == 'Unnamed: 0':
        info_old = info_old.drop(info_old.columns[0], axis=1)
    
    info_save = pd.concat([info_old, info_new])
    info_save.to_csv(info_path, index=False)
except:
    info_new.to_csv(info_path, index=False)

In [16]:
os.rename(model_path_1, local_path + model_name)

In [ ]:
# FIM

In [17]:
# h2o.shutdown(prompt = True)

In [36]:
import ast

info_old = pd.read_csv(info_path)
# ast.literal_eval(info_old['metrics'][0])

In [38]:
json.loads(info_old['varimp'][0])['GLM_1']

{'price_range': 0.1347406085,
 'cs_created_at': 0.2142393112,
 'order_created_at': 0.2639929401,
 'order_total_amount': 0.0204309647,
 'delivery_address_district': 0.1087237397,
 'customer_name': 0.098518271,
 'delivery_address_city': 0.0378153867,
 'minimum_order_value': 0.0010494111,
 'merchant_city': 0.0382788104,
 'merchant_zip_code': 0.0025024681,
 'delivery_time': 0.002581237,
 'origin_platform': 0.0093309757,
 'merchant_timezone': 0.0061754907,
 'delivery_address_state': 0.016167067,
 'merchant_state': 0.0164206886,
 'merchant_longitude': 0.0023088926,
 'takeout_time': 0.0020155215,
 'delivery_address_longitude': 0.0023088926,
 'delivery_address_zip_code': 0.0007867587,
 'value': 0.0007730082,
 'language': 0.0003453972,
 'active': 0.0003801033,
 'order_scheduled': 2.83502e-05,
 'cpf': 0.0001012131,
 'delivery_address_latitude': 0.0002991769,
 'customer_phone_number': 0.0001845882,
 'status_created_at': 0.0001387998,
 'merchant_latitude': 0.0002991769,
 'customer_phone_area': 0.0

In [35]:
info_old['metrics'][0]

"{'mae': '2.4556477488740738', 'rmsle': '0.07654125462553706', 'r2': '0.9545039682546748', 'mean_resid_deviance': '16.398140155357446', 'null_degrees_of_freedom': '7936', 'resid_degrees_of_freedom': '7935', 'null_deviance': '2860758.7243610295', 'residual_deviance': '130152.03841307203'}"

In [44]:
info_old

,id,name,date,h2o_version,seed,target,classification,max_runtime,mse,rmse,train_path,test_path,model_path,columns,varimp,metrics
0,20220123235710,StackedEnsemble_20220123235710,2022-01-23,3.34.0.7,1,average_ticket,False,75,16.39814,4.049462,https://s3model.blob.core.windows.net/modeldata/test_2022012323571...,https://s3model.blob.core.windows.net/modeldata/train_202201232357...,https://s3model.blob.core.windows.net/modeldata/StackedEnsemble_20...,"['price_range', 'cs_created_at', 'order_created_at', 'order_total_...","{""GLM_1"":{""price_range"":0.1347406085,""cs_created_at"":0.2142393112,...","{'mae': '2.4556477488740738', 'rmsle': '0.07654125462553706', 'r2'..."
